In [12]:
import pandas as pd
import numpy as np

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Ler dados
df = pd.read_csv('C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_remove_outliers.csv')

print(df.head())

                      nome             cpf  idade  data           endereco  \
0  Sr. Vitor Gabriel Silva  640.***.***-01     90   NaN  Endereço inválido   
1            Calebe Santos  651.***.***-46     40   NaN  Endereço inválido   
2           Arthur Da Rosa  085.***.***-30     53   NaN  Endereço inválido   
3             Juan Pereira  520.***.***-21     68   NaN  Endereço inválido   
4   Dra. Ana Laura Ribeiro  210.***.***-20     69   NaN  Endereço inválido   

  estado  
0     AP  
1     PB  
2     SE  
3     BA  
4     SP  


### Mascarar dados pessoais (CPF)

In [13]:
df['cpf_mascara'] = df['cpf'].apply(lambda cpf: f'{cpf[:3]}.***.***-{cpf[-2:]}' if isinstance(cpf, str) and len(cpf) == 14 else 'CPF inválido')

### Corrigir Datas

In [14]:
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d', errors='coerce')
data_atual = pd.to_datetime('today')
df['data_atualizada'] = df['data'].where(df['data'] <= data_atual, pd.to_datetime('1900-01-01'))

df['idade_ajustada'] = data_atual.year - df['data_atualizada'].dt.year
df['idade_ajustada'] += ((data_atual.month < df['data_atualizada'].dt.month) |
                         ((data_atual.month == df['data_atualizada'].dt.month) & 
                          (data_atual.day < df['data_atualizada'].dt.day))).astype(int)

### Eliminar idades incoerentes

In [15]:
df.loc[df['idade_ajustada'] > 100, 'idade_ajustada'] = np.nan

### Corrigir campo com múltiplas informações

In [16]:
df['endereco_curto'] = df['endereco'].apply(lambda x: x.split('\\n')[0].strip() if isinstance(x, str) else 'Endereço inválido')
df['bairro'] = df['endereco'].apply(lambda x: x.split('\\n')[1].strip() if isinstance(x, str) and len(x.split('\\n')) > 1 else 'Desconhecido')
df['estado_sigla'] = df['endereco'].apply(lambda x: x.split('\\n')[-1].strip() if isinstance(x, str) and len(x.split('\\n')) > 1 else 'Desconhecido')

### Verificando a formatação do endereço

In [17]:
df['endereco_curto'] = df['endereco_curto'].apply(lambda x: 'Endereço inválido' if len(x) > 50 or len(x) < 5 else x)

### Corrigir dados errôneos de CPF (tamanho errado)

In [7]:
df['cpf'] = df['cpf'].apply(lambda x: x if len(x) == 14 else 'CPF inválido')

In [18]:
# Verificando a formatação do endereço
df['endereco_curto'] = df['endereco_curto'].apply(lambda x: 'Endereço inválido' if len(x) > 50 or len(x) < 5 else x)


### Validar siglas de estados brasileiros

In [19]:
estados_br = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT',
              'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO',
              'RR', 'SC', 'SP', 'SE', 'TO']

df['estado_sigla'] = df['estado_sigla'].str.upper().apply(lambda x: x if x in estados_br else 'Desconhecido')

### Aplicar as correções ao DataFrame final

In [20]:
df['cpf'] = df['cpf_mascara']
df['idade'] = df['idade_ajustada']
df['endereco'] = df['endereco_curto']
df['estado'] = df['estado_sigla']

###  Selecionar colunas finais e salvar

In [23]:
df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'bairro', 'estado']]
df_salvar.to_csv('C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_tratados.csv', index=False)

### Ler o novo CSV para verificação

In [24]:
print("\\nNovo DataFrame: \\n", pd.read_csv('C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_tratados.csv').head())

\nNovo DataFrame: \n                       nome             cpf  idade  data           endereco  \
0  Sr. Vitor Gabriel Silva  640.***.***-01    NaN   NaN  Endereço inválido   
1            Calebe Santos  651.***.***-46    NaN   NaN  Endereço inválido   
2           Arthur Da Rosa  085.***.***-30    NaN   NaN  Endereço inválido   
3             Juan Pereira  520.***.***-21    NaN   NaN  Endereço inválido   
4   Dra. Ana Laura Ribeiro  210.***.***-20    NaN   NaN  Endereço inválido   

         bairro        estado  
0  Desconhecido  Desconhecido  
1  Desconhecido  Desconhecido  
2  Desconhecido  Desconhecido  
3  Desconhecido  Desconhecido  
4  Desconhecido  Desconhecido  


In [25]:
# Apenas substituir se as colunas estiverem corretas
if df['cpf_mascara'].notna().all():
    df['cpf'] = df['cpf_mascara']

if df['idade_ajustada'].notna().sum() > 0:
    df['idade'] = df['idade_ajustada']

if df['endereco_curto'].notna().sum() > 0:
    df['endereco'] = df['endereco_curto']

if df['estado_sigla'].notna().sum() > 0:
    df['estado'] = df['estado_sigla']


In [26]:
print("\nResumo final:")
print(df[['nome', 'cpf', 'idade', 'data', 'endereco', 'bairro', 'estado']].head(10))
print("\nQtd de valores nulos por coluna:\n", df.isnull().sum())



Resumo final:
                      nome             cpf  idade data           endereco  \
0  Sr. Vitor Gabriel Silva  640.***.***-01    NaN  NaT  Endereço inválido   
1            Calebe Santos  651.***.***-46    NaN  NaT  Endereço inválido   
2           Arthur Da Rosa  085.***.***-30    NaN  NaT  Endereço inválido   
3             Juan Pereira  520.***.***-21    NaN  NaT  Endereço inválido   
4   Dra. Ana Laura Ribeiro  210.***.***-20    NaN  NaT  Endereço inválido   
5       Maria Clara Araújo  960.***.***-80    NaN  NaT  Endereço inválido   
6         Vitória Oliveira  302.***.***-83    NaN  NaT  Endereço inválido   
7        Benjamin Oliveira  451.***.***-84    NaN  NaT  Endereço inválido   
8            Pietro Barros  843.***.***-50    NaN  NaT  Endereço inválido   
9        Samuel Cavalcanti  520.***.***-01    NaN  NaT  Endereço inválido   

         bairro        estado  
0  Desconhecido  Desconhecido  
1  Desconhecido  Desconhecido  
2  Desconhecido  Desconhecido  
3  Descon

In [28]:
# Se quiser salvar apenas colunas específicas, como no final do seu projeto:
df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'bairro', 'estado']]

# Salvar em um novo arquivo CSV
df_salvar.to_csv('C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_tratados.csv', index=False)
